In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_long.csv")
#print(data.head())
#data brot:
df= data[data["Warengruppe_6"] == 1].copy()
data_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/2_BaselineModel/02_Franz/weather_data_with_flags.csv")


df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])
df = df.drop(columns=["Datum"])
print(df)



         Umsatz  Temp_Very_Cold  Temp_Cold  Temp_Mild  Temp_Warm  Temp_Hot  \
9042  41.813038               0          0          1          0         0   
9043  16.410876               0          0          1          0         0   
9044  18.574458               0          0          1          0         0   
9045  24.640084               0          0          1          0         0   
9046  43.515366               0          0          1          0         0   
...         ...             ...        ...        ...        ...       ...   
9329  87.471228               0          1          0          0         0   
9330  71.911652               0          1          0          0         0   
9331  84.062223               0          1          0          0         0   
9332  60.981969               0          1          0          0         0   
9333  34.972644               0          1          0          0         0   

      Cloud_Clear  Cloud_Partly_Cloudy  Cloud_Cloudy  Wind_Ligh

In [2]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Umsatz"].sort_values(ascending=False)
print(correlation_with_target)




Umsatz                    1.000000
Weihnachtsmarkt           0.654958
Monat_12                  0.376204
Winter                    0.329182
Temp_Cold                 0.219983
VPI                       0.112808
Samstag                   0.108345
Heimspiel                 0.082065
Weather_Moderate          0.073331
Ship                      0.062118
Number_of_ships           0.044001
Number_of_ships_scaled    0.044001
Temp_Very_Cold            0.038066
Freitag                   0.033373
Wind_ok                   0.031558
Cloud_Clear               0.029886
Cloud_Cloudy              0.020820
Donnerstag                0.020511
Wind_Light                0.018830
Weather_Severe            0.008921
wetter_sehr_schlecht      0.006807
Wind_Moderate             0.002680
Dienstag                  0.001804
Werktag                  -0.020481
Cloud_ok                 -0.020820
Weather_Light_Issues     -0.026879
Wind_Strong              -0.031558
Montag                   -0.033606
Silvester           

In [3]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Umsatz", axis=1)
y = df["Umsatz"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)


Weihnachtsmarkt           0.294963
Monat_12                  0.131507
VPI                       0.123170
Herbst                    0.115644
Winter                    0.115644
Monat_11                  0.085559
Monat_10                  0.035682
Cloud_Clear               0.033371
Wind_Moderate             0.030944
Temp_Mild                 0.029966
Monat_1                   0.026472
zwischen_den_jahren       0.023648
Cloud_Cloudy              0.020330
Cloud_ok                  0.020330
Dienstag                  0.017646
Temp_Very_Cold            0.017137
Temp_Cold                 0.015303
Werktag                   0.014354
Weather_Light_Issues      0.011788
Samstag                   0.010670
Freitag                   0.010551
Sonntag                   0.010500
Number_of_ships           0.006655
Number_of_ships_scaled    0.006368
Ship                      0.003204
Silvester                 0.001142
Donnerstag                0.000000
Wind_Light                0.000000
Mittwoch            

In [4]:
# Features mit geringer linearer und nichtlinearer Korrelation finden
low_linear_corr_features = correlation_with_target[correlation_with_target.abs() < 0.1].index
low_nonlinear_corr_features = mi_series[mi_series < 0.02].index

# Schnittmenge der Features mit beiden Bedingungen
low_corr_features = set(low_linear_corr_features).intersection(low_nonlinear_corr_features)

# Ausgabe der Features
#print("Features mit geringer linearer und nichtlinearer Korrelation:", list(low_corr_features))

# DataFrame erstellen für die Ausgabe in Tabellenform
low_corr_features_table = pd.DataFrame({
    "Feature": list(low_corr_features),
    "Lineare Korrelation": [correlation_with_target[feature] for feature in low_corr_features],
    "Nichtlineare Korrelation": [mi_series[feature] for feature in low_corr_features]
})


# Tabelle nach der linearen Korrelation sortieren
low_corr_features_table = low_corr_features_table.sort_values(by="Lineare Korrelation", ascending = False)

# Ausgabe der sortierten Tabelle
print(low_corr_features_table)



                   Feature  Lineare Korrelation  Nichtlineare Korrelation
11               Heimspiel             0.082065                  0.000000
14        Weather_Moderate             0.073331                  0.000000
18                    Ship             0.062118                  0.003204
9          Number_of_ships             0.044001                  0.006655
6   Number_of_ships_scaled             0.044001                  0.006368
8           Temp_Very_Cold             0.038066                  0.017137
17                 Freitag             0.033373                  0.010551
19                 Wind_ok             0.031558                  0.000000
2               Donnerstag             0.020511                  0.000000
3               Wind_Light             0.018830                  0.000000
5           Weather_Severe             0.008921                  0.000000
1     wetter_sehr_schlecht             0.006807                  0.000000
12                Dienstag            

In [5]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Temp_Cold', 'Temp_Mild', 'Cloud_Cloudy', 'Wind_Light', 'Wind_Moderate', 'Wind_Strong', 'Weather_Light_Issues', 'Weather_Moderate', 'Number_of_ships', 'Ship', 'Herbst', 'Winter', 'Monat_11', 'Monat_12', 'Number_of_ships_scaled', 'Cloud_ok', 'Wind_ok']


In [6]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Umsatz", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


                   Feature       VIF
0           Temp_Very_Cold       inf
1                Temp_Cold       inf
2                Temp_Mild       inf
3                Temp_Warm       NaN
4                 Temp_Hot       NaN
5              Cloud_Clear       inf
6      Cloud_Partly_Cloudy       inf
7             Cloud_Cloudy       inf
8               Wind_Light       inf
9            Wind_Moderate       inf
10             Wind_Strong       inf
11            Weather_Good       inf
12    Weather_Light_Issues       inf
13        Weather_Moderate       inf
14          Weather_Severe       inf
15             KielerWoche       NaN
16                  Montag       inf
17                Dienstag       inf
18                Mittwoch       inf
19              Donnerstag       inf
20                 Freitag       inf
21                 Samstag       inf
22                 Sonntag       inf
23                     VPI  1.147437
24         Number_of_ships       inf
25                    Ship  4.338841
2

In [7]:
# suche miteinander korrelierrende features


In [8]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('Monat_12', 'Weihnachtsmarkt'), ('Monat_11', 'Herbst'), ('Winter', 'Herbst'), ('Monat_12', 'Winter'), ('Ship', 'Number_of_ships'), ('Cloud_ok', 'Cloud_Cloudy'), ('Werktag', 'Sonntag'), ('Monat_11', 'Weihnachtsmarkt'), ('Wind_ok', 'Wind_Strong'), ('Winter', 'Weihnachtsmarkt'), ('Wind_Moderate', 'Wind_Light'), ('Herbst', 'Weihnachtsmarkt'), ('Monat_12', 'Monat_11'), ('Cloud_ok', 'Cloud_Partly_Cloudy'), ('Monat_11', 'Winter'), ('Weather_Moderate', 'Weather_Light_Issues'), ('Cloud_Cloudy', 'Cloud_Partly_Cloudy'), ('Number_of_ships_scaled', 'Ship'), ('Weihnachtsmarkt', 'Umsatz'), ('Heimspiel', 'Samstag'), ('Weather_Light_Issues', 'Weather_Good'), ('Cloud_ok', 'Cloud_Clear'), ('Werktag', 'Samstag'), ('Number_of_ships_scaled', 'Number_of_ships'), ('Cloud_Cloudy', 'Cloud_Clear'), ('Monat_10', 'Temp_Mild'), ('Monat_12', 'Herbst'), ('zwischen_den_jahren', 'is_holiday'), ('Temp_Mild', 'Temp_Cold')}
